# Explication 1-1 pour le dataset Breast Cancer

In [27]:
import pandas as pd
from gurobipy import *

# Chargement des données

df = pd.read_csv("breastcancer_processed.csv")

features = [
    "ClumpThickness",
    "UniformityOfCellSize",
    "UniformityOfCellShape",
    "MarginalAdhesion",
    "SingleEpithelialCellSize",
    "BareNuclei",
    "BlandChromatin",
    "NormalNucleoli",
    "Mitoses"
]

# Classe cible : Malignant = 1
y = 1 - df["Benign"]
X = df[features]

# Poids

weights = X.apply(lambda col: col.corr(y)).to_dict()

# Instance à expliquer

i = 0  # patient
x = X.iloc[i]
mu = X.mean()

# Calcul des deltas

deltas = {f: weights[f] * (x[f] - mu[f]) for f in features}

# Pros et Cons
pros = [f for f, v in deltas.items() if v > 0]
cons = [f for f, v in deltas.items() if v < 0]

print(f"Pros: {pros}")
print(f"Cons: {cons}")
print(f"Deltas: {deltas}")

# Calcul des paires 1-1 valides

valid_pairs = [(p, c) for p in pros for c in cons if deltas[p] + deltas[c] > 0]
print(f"Nombre de paires valides identifiées : {len(valid_pairs)}")

# Modélisation Gurobi 1-1

m = Model("Explication_1_1_BreastCancer")

c_vars = { (p,c): m.addVar(vtype=GRB.BINARY, name=f"c_{p}_{c}") for p, c in valid_pairs }
m.update()

# Chaque Con doit être couvert exactement une fois
for c_feat in cons:
    m.addConstr(quicksum(c_vars[(p,c_feat)] for p in pros if (p,c_feat) in valid_pairs) == 1, name=f"Cover_{c_feat}")

# Chaque Pro utilisé au maximum une fois
for p_feat in pros:
    m.addConstr(quicksum(c_vars[(p_feat,c)] for c in cons if (p_feat,c) in valid_pairs) <= 1, name=f"Unique_{p_feat}")

# Objectif : faisabilité
m.setObjective(0, GRB.MINIMIZE)
m.params.outputflag = 0
m.optimize()

# Analyse des résultats

if m.status == GRB.OPTIMAL:
    print("\nSolution 1-1 trouvée :")
    for p, c in valid_pairs:
        if c_vars[(p,c)].x > 0.5:
            net_val = deltas[p] + deltas[c]
            print(f"Trade-off Pro '{p}' (Δ={deltas[p]:+.4f}) / Con '{c}' (Δ={deltas[c]:+.4f}) -> Net: {net_val:+.4f}")

elif m.status == GRB.INFEASIBLE:
    print("\nAucune explication 1-1 trouvée.")


Pros: ['UniformityOfCellSize', 'UniformityOfCellShape', 'MarginalAdhesion', 'SingleEpithelialCellSize', 'BareNuclei', 'BlandChromatin', 'NormalNucleoli', 'Mitoses']
Cons: ['ClumpThickness']
Deltas: {'ClumpThickness': np.float64(-0.3987334728092868), 'UniformityOfCellSize': np.float64(1.7653840695625356), 'UniformityOfCellShape': np.float64(1.8206749690384447), 'MarginalAdhesion': np.float64(1.2926320195206518), 'SingleEpithelialCellSize': np.float64(0.8528224423288597), 'BareNuclei': np.float64(2.093477931578111), 'BlandChromatin': np.float64(0.3374834210514831), 'NormalNucleoli': np.float64(1.343705371767472), 'Mitoses': np.float64(0.2554327138706111)}
Nombre de paires valides identifiées : 6

Solution 1-1 trouvée :
Trade-off Pro 'UniformityOfCellSize' (Δ=+1.7654) / Con 'ClumpThickness' (Δ=-0.3987) -> Net: +1.3667


Dans le modèle 1-1, chaque con doit être compensé par exactement un pro.

Le modèle identifie toutes les paires pro–con dont la somme des deltas > 0, car seules ces paires peuvent neutraliser le con.

Il y a 6 pros dont l’effet positif est suffisant pour compenser 'ClumpThickness'.

Le patient présente un 'ClumpThickness' légèrement en faveur d’une tumeur bénigne, mais l’uniformité de la taille des cellules 'UniformityOfCellSize' compense largement cet effet. Au final, le profil global reste typique d’une tumeur maligne.

# Explication 1-m pour le dataset Breast Cancer

In [28]:
import pandas as pd
from gurobipy import *

# Chargement des données

df = pd.read_csv("breastcancer_processed.csv")

features = [
    "ClumpThickness",
    "UniformityOfCellSize",
    "UniformityOfCellShape",
    "MarginalAdhesion",
    "SingleEpithelialCellSize",
    "BareNuclei",
    "BlandChromatin",
    "NormalNucleoli",
    "Mitoses"
]

# Classe cible : Malignant = 1
y = 1 - df["Benign"]
X = df[features]

# Poids

weights = X.apply(lambda col: col.corr(y)).to_dict()

# Instance à expliquer

i = 0  # patient
x = X.iloc[i]
mu = X.mean()

# Calcul des deltas

deltas = {}
for f in features:
    deltas[f] = weights[f] * (x[f] - mu[f])

pros = [f for f in deltas if deltas[f] > 0]
cons = [f for f in deltas if deltas[f] < 0]

print("Pros :", pros)
print("Cons :", cons)

# Modèle Gurobi 1–m

m = Model("BreastCancer_Explanation_1_m")
x_var = m.addVars(pros, cons, vtype=GRB.BINARY, name="x")
m.update()

# Contraintes

# Chaque Con est couvert exactement une fois
for c in cons:
    m.addConstr(quicksum(x_var[p, c] for p in pros) == 1, name=f"Cover_{c}")

# Validité des trade-offs (1–m)
epsilon = 1e-6
for p in pros:
    m.addConstr(
        deltas[p] + quicksum(deltas[c] * x_var[p, c] for c in cons) >= epsilon,
        name=f"Valid_{p}"
    )

# Résolution

m.setObjective(0, GRB.MINIMIZE)
m.params.outputflag = 0
m.optimize()

# Analyse

if m.status == GRB.OPTIMAL:
    print("\n")
    print("Explication (1–m) trouvée\n")

    for p in pros:
        covered = [c for c in cons if x_var[p, c].x > 0.5]
        if covered:
            net = deltas[p] + sum(deltas[c] for c in covered)
            print(f"Pro {p} (Δ = {deltas[p]:+.4f})")
            print(f"  compense {covered}")
            print(f"  bilan net = {net:+.4f}\n")

elif m.status == GRB.INFEASIBLE:
    print("Aucune explication (1–m) possible.")


Pros : ['UniformityOfCellSize', 'UniformityOfCellShape', 'MarginalAdhesion', 'SingleEpithelialCellSize', 'BareNuclei', 'BlandChromatin', 'NormalNucleoli', 'Mitoses']
Cons : ['ClumpThickness']


Explication (1–m) trouvée

Pro UniformityOfCellSize (Δ = +1.7654)
  compense ['ClumpThickness']
  bilan net = +1.3667



Les cons vont dans le sens de la tumeur benigne et les pros dans le sens de la tumeur maligne.
Pour le premier patient, seul le critère 'ClumpThickness' va dans le sens de la tumeur benigne.
Grâce à l'algorithme on trouve que 'UniformityOfCellSize' est suffisament fort pour neutraliser 'ClumpThickness'.

Etant donné que notre solveur cherche n'importe quelle solution faisable, 'UniformityOfCellSize' est le pro le plus fort.
On obtient le choix minimal et rationnel.

On peut reformuler ce résultat comme suit :
Même si l’épaisseur des amas cellulaires est compatible avec une tumeur bénigne, l’uniformité anormale de la taille des cellules est suffisamment marquée pour justifier une prédiction maligne.

# Explication mixte m-1 pour le dataset Breast Cancer

In [29]:
import pandas as pd
from gurobipy import *

# Chargement des données

df = pd.read_csv("breastcancer_processed.csv")

features = [
    "ClumpThickness",
    "UniformityOfCellSize",
    "UniformityOfCellShape",
    "MarginalAdhesion",
    "SingleEpithelialCellSize",
    "BareNuclei",
    "BlandChromatin",
    "NormalNucleoli",
    "Mitoses"
]

# Classe cible : Malignant = 1
y = 1 - df["Benign"]
X = df[features]

# Poids (importance globale)

weights = X.apply(lambda col: col.corr(y)).to_dict()

# Instance à expliquer

i = 0  # patient
x = X.iloc[i]
mu = X.mean()

# Calcul des deltas

deltas = {f: weights[f] * (x[f] - mu[f]) for f in features}

pros = [f for f in deltas if deltas[f] > 0]
cons = [f for f in deltas if deltas[f] < 0]

print("Pros :", pros)
print("Cons :", cons)
print("Deltas :", deltas)

# Modèle Gurobi m–1

m = Model("BreastCancer_Explanation_m_1")

y_var = m.addVars(pros, cons, vtype=GRB.BINARY, name="y")
m.update()

# Contraintes m–1

# Chaque Pro utilisé au plus une fois
for p in pros:
    m.addConstr(quicksum(y_var[p, c] for c in cons) <= 1, name=f"Unique_{p}")

# Chaque Con doit être compensé par un ou plusieurs Pros
epsilon = 1e-6
for c in cons:
    m.addConstr(
        deltas[c] + quicksum(deltas[p] * y_var[p, c] for p in pros) >= epsilon,
        name=f"Valid_Group_for_{c}"
    )

# Résolution

m.setObjective(0, GRB.MINIMIZE)
m.params.outputflag = 0
m.optimize()

# Analyse des résultats

if m.status == GRB.OPTIMAL:
    print("\n")
    print("Explication (m–1) trouvée\n")
    for c in cons:
        assigned_pros = [p for p in pros if y_var[p, c].x > 0.5]
        sum_pros = sum(deltas[p] for p in assigned_pros)
        net_value = deltas[c] + sum_pros
        print(f"Argument Contre {c} (Δ={deltas[c]:+.4f}) compensé par : {assigned_pros}")
        print(f"  cumul Pros = {sum_pros:+.4f}, bilan net = {net_value:+.4f}\n")

elif m.status == GRB.INFEASIBLE:
    print("Aucune explication (m–1) possible.")

Pros : ['UniformityOfCellSize', 'UniformityOfCellShape', 'MarginalAdhesion', 'SingleEpithelialCellSize', 'BareNuclei', 'BlandChromatin', 'NormalNucleoli', 'Mitoses']
Cons : ['ClumpThickness']
Deltas : {'ClumpThickness': np.float64(-0.3987334728092868), 'UniformityOfCellSize': np.float64(1.7653840695625356), 'UniformityOfCellShape': np.float64(1.8206749690384447), 'MarginalAdhesion': np.float64(1.2926320195206518), 'SingleEpithelialCellSize': np.float64(0.8528224423288597), 'BareNuclei': np.float64(2.093477931578111), 'BlandChromatin': np.float64(0.3374834210514831), 'NormalNucleoli': np.float64(1.343705371767472), 'Mitoses': np.float64(0.2554327138706111)}


Explication (m–1) trouvée

Argument Contre ClumpThickness (Δ=-0.3987) compensé par : ['UniformityOfCellSize', 'UniformityOfCellShape', 'MarginalAdhesion', 'SingleEpithelialCellSize', 'BareNuclei', 'BlandChromatin', 'NormalNucleoli', 'Mitoses']
  cumul Pros = +9.7616, bilan net = +9.3629




Ici, chaque con doit être compensé par un ou plusieurs pros et chaque pro peut être utilisé au plus une fois.

Le seul con est 'ClumpThickness', tous les autres critères vont servir à le compenser.

La somme des Pros (+9.7616) est largement supérieure au delta négatif (-0.3987), donc la contrainte est satisfaite.

Même si ClumpThickness est contre, l’ensemble des autres variables (pros) compense largement.


# Explication mixte (1-m et m-1) pour le dataset Breast Cancer

In [30]:
import pandas as pd
from gurobipy import *

# Chargement des données

df = pd.read_csv("breastcancer_processed.csv")

features = [
    "ClumpThickness",
    "UniformityOfCellSize",
    "UniformityOfCellShape",
    "MarginalAdhesion",
    "SingleEpithelialCellSize",
    "BareNuclei",
    "BlandChromatin",
    "NormalNucleoli",
    "Mitoses"
]

# Classe cible : Malignant = 1
y = 1 - df["Benign"]
X = df[features]

# Poids

weights = X.apply(lambda col: col.corr(y)).to_dict()

# Instance à expliquer

i = 0  # patient
x = X.iloc[i]
mu = X.mean()

# Calcul des contributions

deltas = {f: weights[f] * (x[f] - mu[f]) for f in features}
pros = [f for f, v in deltas.items() if v > 0]
cons = [f for f, v in deltas.items() if v < 0]

print(f"Instance {i}")
print("Pros :", pros)
print("Cons :", cons)
print("Deltas :", deltas)

# Modèle mixte 1-m et m-1

m = Model("BreastCancer_Explanation_Mixte")

# Variables pivots
z_1m = m.addVars(pros, vtype=GRB.BINARY, name="z_1m")  # Pro pivot 1-m
z_m1 = m.addVars(cons, vtype=GRB.BINARY, name="z_m1")  # Con pivot m-1

# Variables de relations
v = m.addVars(pros, cons, vtype=GRB.BINARY, name="v")  # p couvre c en 1-m
w = m.addVars(pros, cons, vtype=GRB.BINARY, name="w")  # p aide c en m-1

m.update()

# Contraintes mixtes

epsilon = 1e-6
BigM = 1000

# Chaque con est soit pivot m-1, soit couvert par un pro en 1-m
for c in cons:
    m.addConstr(z_m1[c] + quicksum(v[p, c] for p in pros) == 1, name=f"CoverCons_{c}")

# Chaque pro est soit pivot 1-m, soit utilisé en m-1, ou rien
for p in pros:
    m.addConstr(z_1m[p] + quicksum(w[p, c] for c in cons) <= 1, name=f"UniquePro_{p}")

# Consistance des liens
for p in pros:
    for c in cons:
        m.addConstr(v[p, c] <= z_1m[p])
        m.addConstr(w[p, c] <= z_m1[c])

# Validité des groupes
for p in pros:
    m.addConstr(deltas[p] + quicksum(deltas[c] * v[p, c] for c in cons) >= epsilon - BigM * (1 - z_1m[p]))
for c in cons:
    m.addConstr(deltas[c] + quicksum(deltas[p] * w[p, c] for p in pros) >= epsilon - BigM * (1 - z_m1[c]))

# Objectif

m.setObjective(quicksum(z_1m[p] for p in pros) + quicksum(z_m1[c] for c in cons), GRB.MINIMIZE)

m.params.outputflag = 0
m.optimize()

# Analyse des résultats

if m.status == GRB.OPTIMAL:
    print("\n")
    print("Solution Mixte trouvée :\n")
    
    # Trade-offs 1-m
    for p in pros:
        if z_1m[p].x > 0.5:
            my_cons = [c for c in cons if v[p, c].x > 0.5]
            val = deltas[p] + sum(deltas[c] for c in my_cons)
            print(f"[Type 1-m] Pro '{p}' (Δ={deltas[p]:+.4f}) couvre Cons {my_cons} (Net: {val:+.4f})")

    # Trade-offs m-1
    for c in cons:
        if z_m1[c].x > 0.5:
            my_pros = [p for p in pros if w[p, c].x > 0.5]
            val = deltas[c] + sum(deltas[p] for p in my_pros)
            print(f"[Type m-1] Con '{c}' (Δ={deltas[c]:+.4f}) compensé par Pros {my_pros} (Net: {val:+.4f})")

elif m.status == GRB.INFEASIBLE:
    print("Aucune explication mixte possible.")


Instance 0
Pros : ['UniformityOfCellSize', 'UniformityOfCellShape', 'MarginalAdhesion', 'SingleEpithelialCellSize', 'BareNuclei', 'BlandChromatin', 'NormalNucleoli', 'Mitoses']
Cons : ['ClumpThickness']
Deltas : {'ClumpThickness': np.float64(-0.3987334728092868), 'UniformityOfCellSize': np.float64(1.7653840695625356), 'UniformityOfCellShape': np.float64(1.8206749690384447), 'MarginalAdhesion': np.float64(1.2926320195206518), 'SingleEpithelialCellSize': np.float64(0.8528224423288597), 'BareNuclei': np.float64(2.093477931578111), 'BlandChromatin': np.float64(0.3374834210514831), 'NormalNucleoli': np.float64(1.343705371767472), 'Mitoses': np.float64(0.2554327138706111)}


Solution Mixte trouvée :

[Type m-1] Con 'ClumpThickness' (Δ=-0.3987) compensé par Pros ['BlandChromatin', 'Mitoses'] (Net: +0.1942)


Le modèle a trouvé une explication de type m-1.

Le seul con 'ClumpThickness' est compensé par deux pros : 'BlandChromatin', 'Mitoses'.
Le modèle a choisi de ne pas utiliser tous les pros, juste ceux qui suffisent pour compenser 'ClumpThickness'.
Pour ce patient, presque toutes les features favorisent Maligne, sauf 'ClumpThickness'. Même si 'ClumpThickness' est contre, il est largement neutralisé par 'BlandChromatin' et 'Mitoses', donc la décision finale reste positive.